In [1]:

import csv
import pandas as pd
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
df = pd.read_csv('Capstone updated 1219.csv', parse_dates = ['latest_transaction','latest_membership_expiration','latest_listen_date','registration_init_time'])
df

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,latest_transaction,latest_membership_expiration,is_cancel,avg_membership_duration,avg_time_between_transactions,latest_listen_date,num_unq,total_secs,percent_25,percent_50,percent_100,city,registered_via,registration_init_time
0,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-19,0.0,49.5,15.5,2017-03-31,43.0,115411.260,0.075838,0.021164,0.876543,15.0,9.0,2010-11-18
1,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-23,0.0,53.5,15.5,2017-03-30,52.0,36636.638,0.317597,0.077253,0.562232,22.0,9.0,2012-01-22
2,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,0,14.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-04,0.0,34.5,15.5,2017-03-25,25.0,10243.065,0.090909,0.072727,0.763636,13.0,3.0,2014-04-21
3,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-21,0.0,51.5,15.5,2017-03-28,80.0,292650.977,0.157303,0.024345,0.808989,13.0,9.0,2006-02-18
4,++6P09mCSJSh+Ft2pvZ0FWTrtcI3v1A7h3/coh8dBKw=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-13,0.0,43.5,15.5,2017-03-25,7.0,2994.947,0.000000,0.142857,0.857143,22.0,9.0,2010-09-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725717,zzx1qgX8R9EmT8PAn4/KyQ6s7PuLYTfR8kUCABiCW9E=,0,41.0,30.0,99.0,99.0,1.0,2017-03-31,2017-04-30,0.0,30.0,0.0,2017-03-24,38.0,27727.142,0.057554,0.007194,0.913669,6.0,9.0,2014-10-14
725718,zzx4hKiyR9XFEGAr7SAjcCPbKJCZ+IqegWL7dPjPwZk=,0,41.0,30.0,99.0,99.0,1.0,2017-03-18,2017-04-19,0.0,32.0,0.0,2017-03-26,45.0,51394.898,0.095238,0.035714,0.861111,1.0,7.0,2011-08-23
725719,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,0,41.0,30.0,99.0,99.0,1.0,2017-03-26,2017-04-26,0.0,31.0,0.0,2017-03-10,40.0,13172.171,0.033333,0.000000,0.966667,4.0,7.0,2012-03-06
725720,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,1,36.0,30.0,180.0,180.0,0.0,2017-02-28,2017-04-01,0.0,32.0,0.0,2017-03-31,56.0,268039.763,0.094642,0.048017,0.825331,5.0,3.0,2015-04-30


In [4]:
df.dtypes

msno                                     object
is_churn                                  int64
payment_method_id                       float64
payment_plan_days                       float64
plan_list_price                         float64
actual_amount_paid                      float64
is_auto_renew                           float64
latest_transaction               datetime64[ns]
latest_membership_expiration     datetime64[ns]
is_cancel                               float64
avg_membership_duration                 float64
avg_time_between_transactions           float64
latest_listen_date               datetime64[ns]
num_unq                                 float64
total_secs                              float64
percent_25                              float64
percent_50                              float64
percent_100                             float64
city                                    float64
registered_via                          float64
registration_init_time           datetim

In [5]:
df['latest_transaction_year'] = df['latest_transaction'].dt.year
df['latest_transaction_month'] = df['latest_transaction'].dt.month
df['latest_transaction_week'] = df['latest_transaction'].dt.week
df['latest_transaction_day'] = df['latest_transaction'].dt.day

C:\Users\marsh\AppData\Local\Temp/ipykernel_13868/2407976660.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['latest_transaction_week'] = df['latest_transaction'].dt.week


In [6]:
df

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,latest_transaction,latest_membership_expiration,is_cancel,avg_membership_duration,avg_time_between_transactions,latest_listen_date,num_unq,total_secs,percent_25,percent_50,percent_100,city,registered_via,registration_init_time,latest_transaction_year,latest_transaction_month,latest_transaction_week,latest_transaction_day
0,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-19,0.0,49.5,15.5,2017-03-31,43.0,115411.260,0.075838,0.021164,0.876543,15.0,9.0,2010-11-18,2017,3,13,31
1,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-23,0.0,53.5,15.5,2017-03-30,52.0,36636.638,0.317597,0.077253,0.562232,22.0,9.0,2012-01-22,2017,3,13,31
2,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,0,14.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-04,0.0,34.5,15.5,2017-03-25,25.0,10243.065,0.090909,0.072727,0.763636,13.0,3.0,2014-04-21,2017,3,13,31
3,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-21,0.0,51.5,15.5,2017-03-28,80.0,292650.977,0.157303,0.024345,0.808989,13.0,9.0,2006-02-18,2017,3,13,31
4,++6P09mCSJSh+Ft2pvZ0FWTrtcI3v1A7h3/coh8dBKw=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-13,0.0,43.5,15.5,2017-03-25,7.0,2994.947,0.000000,0.142857,0.857143,22.0,9.0,2010-09-21,2017,3,13,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725717,zzx1qgX8R9EmT8PAn4/KyQ6s7PuLYTfR8kUCABiCW9E=,0,41.0,30.0,99.0,99.0,1.0,2017-03-31,2017-04-30,0.0,30.0,0.0,2017-03-24,38.0,27727.142,0.057554,0.007194,0.913669,6.0,9.0,2014-10-14,2017,3,13,31
725718,zzx4hKiyR9XFEGAr7SAjcCPbKJCZ+IqegWL7dPjPwZk=,0,41.0,30.0,99.0,99.0,1.0,2017-03-18,2017-04-19,0.0,32.0,0.0,2017-03-26,45.0,51394.898,0.095238,0.035714,0.861111,1.0,7.0,2011-08-23,2017,3,11,18
725719,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,0,41.0,30.0,99.0,99.0,1.0,2017-03-26,2017-04-26,0.0,31.0,0.0,2017-03-10,40.0,13172.171,0.033333,0.000000,0.966667,4.0,7.0,2012-03-06,2017,3,12,26
725720,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,1,36.0,30.0,180.0,180.0,0.0,2017-02-28,2017-04-01,0.0,32.0,0.0,2017-03-31,56.0,268039.763,0.094642,0.048017,0.825331,5.0,3.0,2015-04-30,2017,2,9,28


In [7]:
df['latest_membership_expiration_year'] = df['latest_membership_expiration'].dt.year
df['latest_membership_expiration_month'] = df['latest_membership_expiration'].dt.month
df['latest_membership_expiration_week'] = df['latest_membership_expiration'].dt.week
df['latest_membership_expiration_day'] = df['latest_membership_expiration'].dt.day

C:\Users\marsh\AppData\Local\Temp/ipykernel_13868/2489746929.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['latest_membership_expiration_week'] = df['latest_membership_expiration'].dt.week


In [9]:
df['latest_listen_date_year'] = df['latest_listen_date'].dt.year
df['latest_listen_date_month'] = df['latest_listen_date'].dt.month
df['latest_listen_date_week'] = df['latest_listen_date'].dt.week
df['latest_listen_date_day'] = df['latest_listen_date'].dt.day

C:\Users\marsh\AppData\Local\Temp/ipykernel_13868/2240293902.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['latest_listen_date_week'] = df['latest_listen_date'].dt.week


In [10]:
df['registration_init_time_year'] = df['registration_init_time'].dt.year
df['registration_init_time_month'] = df['registration_init_time'].dt.month
df['registration_init_time_week'] = df['registration_init_time'].dt.week
df['registration_init_time_day'] = df['registration_init_time'].dt.day

C:\Users\marsh\AppData\Local\Temp/ipykernel_13868/2671360494.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['registration_init_time_week'] = df['registration_init_time'].dt.week


In [19]:
df = df.drop(columns = ['latest_transaction','latest_membership_expiration','latest_listen_date'])

In [20]:
y = df['is_churn']
X = df.drop(columns = 'is_churn')

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .30, random_state = 42)

In [22]:
X

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,avg_membership_duration,avg_time_between_transactions,num_unq,total_secs,percent_25,percent_50,percent_100,city,registered_via,registration_init_time,latest_transaction_year,latest_transaction_month,latest_transaction_week,latest_transaction_day,latest_membership_expiration_year,latest_membership_expiration_month,latest_membership_expiration_week,latest_membership_expiration_day,latest_listen_date_year,latest_listen_date_month,latest_listen_date_week,latest_listen_date_day,registration_init_time_year,registration_init_time_month,registration_init_time_week,registration_init_time_day
0,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39.0,30.0,149.0,149.0,1.0,0.0,49.5,15.5,43.0,115411.260,0.075838,0.021164,0.876543,15.0,9.0,2010-11-18,2017,3,13,31,2017,5,20,19,2017,3,13,31,2010,11,46,18
1,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,39.0,30.0,149.0,149.0,1.0,0.0,53.5,15.5,52.0,36636.638,0.317597,0.077253,0.562232,22.0,9.0,2012-01-22,2017,3,13,31,2017,5,21,23,2017,3,13,30,2012,1,3,22
2,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,14.0,30.0,149.0,149.0,1.0,0.0,34.5,15.5,25.0,10243.065,0.090909,0.072727,0.763636,13.0,3.0,2014-04-21,2017,3,13,31,2017,5,18,4,2017,3,12,25,2014,4,17,21
3,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,39.0,30.0,149.0,149.0,1.0,0.0,51.5,15.5,80.0,292650.977,0.157303,0.024345,0.808989,13.0,9.0,2006-02-18,2017,3,13,31,2017,5,20,21,2017,3,13,28,2006,2,7,18
4,++6P09mCSJSh+Ft2pvZ0FWTrtcI3v1A7h3/coh8dBKw=,39.0,30.0,149.0,149.0,1.0,0.0,43.5,15.5,7.0,2994.947,0.000000,0.142857,0.857143,22.0,9.0,2010-09-21,2017,3,13,31,2017,5,19,13,2017,3,12,25,2010,9,38,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725717,zzx1qgX8R9EmT8PAn4/KyQ6s7PuLYTfR8kUCABiCW9E=,41.0,30.0,99.0,99.0,1.0,0.0,30.0,0.0,38.0,27727.142,0.057554,0.007194,0.913669,6.0,9.0,2014-10-14,2017,3,13,31,2017,4,17,30,2017,3,12,24,2014,10,42,14
725718,zzx4hKiyR9XFEGAr7SAjcCPbKJCZ+IqegWL7dPjPwZk=,41.0,30.0,99.0,99.0,1.0,0.0,32.0,0.0,45.0,51394.898,0.095238,0.035714,0.861111,1.0,7.0,2011-08-23,2017,3,11,18,2017,4,16,19,2017,3,12,26,2011,8,34,23
725719,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,41.0,30.0,99.0,99.0,1.0,0.0,31.0,0.0,40.0,13172.171,0.033333,0.000000,0.966667,4.0,7.0,2012-03-06,2017,3,12,26,2017,4,17,26,2017,3,10,10,2012,3,10,6
725720,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,36.0,30.0,180.0,180.0,0.0,0.0,32.0,0.0,56.0,268039.763,0.094642,0.048017,0.825331,5.0,3.0,2015-04-30,2017,2,9,28,2017,4,13,1,2017,3,13,31,2015,4,18,30
